*Note: To use the Watson NLP library, this notebook must be run in IBM Watson Studio with environment: "Default Python 3.8 + Watson NLP XS (beta)".*

# Color coding sticky notes by sentiment

This notebook demonstrates how to analyze the sentiment of sticky contents using the [Watson NLP library](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp.html).

The sample mural for this notebook is here: [Sample mural](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-04_sentiment.json)

You can create that mural using the notebook here: [Mural creating notebook](https://github.com/spackows/MURAL-API-Samples/blob/main/notebooks/sample-01_create-a-mural.ipynb)

The sample mural for this notebook (before the color coding) looks like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-04_sentiment.png" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [13]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [37]:
g_auth_token = ""

## Step 3: Read the widgets from the mural

In this sample, it's assumed that all widgets in the mural are sticky notes.

In [15]:
import requests
import json

def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [16]:
widgets_arr = listWidgets( g_mural_id, g_auth_token )
for widget in widgets_arr:
    print( widget["text"] )

I love winter!
I don't do well in the heat
Fall is colourful
Spring makes me feel optimistic
I hate the cold
Fall leaves are beautiful
Nothing is as soothing as drinking hot chocolate after being out in the cold
Swimming in the summer makes me so happy
I feel sad watching everything die off in the fall
summer is for relaxing
spring and fall are shorter
seasons bring a change of pace
We go south for the winter every year


## Step 4: Analyze sentiment of sticky contents

In [17]:
import watson_nlp
from watson_nlp.toolkit import predict_document_sentiment

# Load built-in Syntax model and Sentiment model for English
syntax_model = watson_nlp.load( watson_nlp.download( "syntax_izumo_en_stock" ) )
sentiment_model = watson_nlp.load( watson_nlp.download( "sentiment_sentence-bert_multi_stock" ) )

In [18]:
import pandas as pd
import re

def analyzeComment( comment ):
    syntax_prediction = syntax_model.run( comment )
    sentiment_result  = sentiment_model.run_batch( syntax_result.get_sentence_texts(), syntax_result.sentences )
    document_sentiment = predict_document_sentiment( sentiment_result, sentiment_model.class_idxs )
    sentiment_dict = document_sentiment.to_dict()
    sentiment_dict["label"] = re.sub( r"^.*_", "", sentiment_dict["label"].lower() ).title()
    return sentiment_dict

def analyzeStickies( stickies_arr ):
    sentiment_results = []
    for sticky in stickies_arr:
        if "text" in sticky and re.match( "\S", sticky["text"] ):
            sentiment = analyzeComment( sticky["text"] )
            sentiment_results.append( [ sticky["id"], sentiment["score"], sentiment["label"], sticky["text"] ] )
    df = pd.DataFrame( sentiment_results, columns=[ "ID", "SCORE", "SENTIMENT", "TEXT" ] )
    return df

In [19]:
df = analyzeStickies( widgets_arr )
df

,ID,SCORE,SENTIMENT,TEXT
0,08236fbb-198e-4c25-b6b9-7df30ed73589,0.982425,Positive,I love winter!
1,6956c909-f5bd-45ea-9a84-9dd12a4b0980,-0.686673,Negative,I don't do well in the heat
2,8b172423-4cfd-4f5e-87b0-c1492c642178,0.000000,Neutral,Fall is colourful
3,0-1651021491666,0.997671,Positive,Spring makes me feel optimistic
4,0-1651021504402,-0.898896,Negative,I hate the cold
5,0-1651021518502,0.981306,Positive,Fall leaves are beautiful
6,0-1651021530675,0.247025,Positive,Nothing is as soothing as drinking hot chocola...
7,0-1651021589987,0.993842,Positive,Swimming in the summer makes me so happy
8,0-1651021606777,-0.996813,Negative,I feel sad watching everything die off in the ...
9,0-1651021656085,0.747447,Positive,summer is for relaxing


## Step 5: Color code stickies based on sentiment

In [25]:
def colorSticky( sticky_id, color, mural_id, auth_token ):
    # https://developers.mural.co/public/reference/updatestickynote
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note/" + sticky_id
    headers = { "Content-Type" : "application/json", "Accept" : "vnd.mural.preview", "Authorization" : "Bearer " + auth_token }
    data = json.dumps( { "style" : { "backgroundColor" : color } } )
    response = requests.request( "PATCH", url, headers=headers, data=data )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + sticky_id + " ]  " + msg )

def colorCodeStickies( stickies_arr, sentiment_df, mural_id, auth_token ):
    colors = { "Positive" : "#046307FF",  # green
               "Neutral"  : "#FFA500FF",  # orange
               "Negative" : "#0000CDFF" } # blue
    for sticky in stickies_arr:
        sentiment = list( sentiment_df[ sentiment_df["ID"] == sticky["id"] ]["SENTIMENT"] )[0]
        colorSticky( sticky["id"], colors[ sentiment ], mural_id, auth_token )

In [38]:
import time
time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the colors change
# ...

colorCodeStickies( widgets_arr, df, g_mural_id, g_auth_token )

Like this:

<img src="https://github.com/spackows/MURAL-API-Samples/blob/main/images/sample-05_color-code-by-sentiment.gif?raw=true" width="50%" title="Colors changing" />